In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D,Dense,Dropout,Flatten
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

In [3]:
train=pd.read_csv('train.csv')
train.head()

,filename,label
0,0.png,4
1,1.png,9
2,2.png,1
3,3.png,7
4,4.png,3


In [4]:
from os.path import join
train_paths='Images/train'
val_paths='Images/val'
filenames=train['filename'].tolist()
img_paths = [join(train_paths, filename) for filename in filenames]

In [5]:
from tensorflow import keras
y=keras.utils.to_categorical(train['label'],10)

In [6]:
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array

image_size = 28

def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    output = preprocess_input(img_array)
    return(output)

In [7]:
x=read_and_prep_images(img_paths)

In [8]:
x.shape

(49000, 28, 28, 3)

In [9]:
model=Sequential()
img_rows=28
img_cols=28

In [10]:
model.add(Conv2D(20, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 3)))
model.add(Dropout(0.5))
model.add(Conv2D(20,kernel_size=3,activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(18,kernel_size=2,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 20)        560       
_________________________________________________________________
dropout (Dropout)            (None, 26, 26, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 20)        3620      
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 18)        1458      
_________________________________________________________________
dropout_2 (Dropout)          (None, 23, 23, 18)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9522)              0

In [11]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
model.fit(x,y,batch_size=128,epochs=8,validation_split=0.2)

Train on 39200 samples, validate on 9800 samples
Epoch 1/8
39200/39200 [==============================] - 66s 2ms/sample - loss: 0.3731 - accuracy: 0.9006 - val_loss: 0.2013 - val_accuracy: 0.9520
Epoch 2/8
39200/39200 [==============================] - 62s 2ms/sample - loss: 0.3140 - accuracy: 0.9120 - val_loss: 0.1942 - val_accuracy: 0.9545
Epoch 3/8
39200/39200 [==============================] - 63s 2ms/sample - loss: 0.2683 - accuracy: 0.9219 - val_loss: 0.1536 - val_accuracy: 0.9604
Epoch 4/8
39200/39200 [==============================] - 63s 2ms/sample - loss: 0.2403 - accuracy: 0.9289 - val_loss: 0.1515 - val_accuracy: 0.9627
Epoch 5/8
39200/39200 [==============================] - 67s 2ms/sample - loss: 0.2278 - accuracy: 0.9317 - val_loss: 0.1355 - val_accuracy: 0.9629
Epoch 6/8
39200/39200 [==============================] - 65s 2ms/sample - loss: 0.2067 - accuracy: 0.9379 - val_loss: 0.1338 - val_accuracy: 0.9684
Epoch 7/8
39200/39200 [==============================] - 75s 2m

In [13]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [15]:
test=pd.read_csv('Test.csv')
test.head()

,filename
0,49000.png
1,49001.png
2,49002.png
3,49003.png
4,49004.png


In [16]:
from os.path import join
train_paths='Images/test'
filenames=test['filename'].tolist()
img_paths = [join(train_paths, filename) for filename in filenames]
x_test=read_and_prep_images(img_paths)

In [25]:
y_hat=model.predict(x_test)
y_hat

array([[1.6853848e-05, 8.8601411e-07, 7.6560964e-06, ..., 1.3774693e-05,
        6.6147346e-05, 1.8358938e-03],
       [9.4154334e-01, 2.0833233e-05, 5.6123924e-02, ..., 7.2062547e-05,
        9.2775567e-04, 3.0446099e-04],
       [9.5142174e-04, 2.2011209e-06, 7.8611756e-03, ..., 2.6001697e-02,
        2.7671058e-04, 9.4995630e-01],
       ...,
       [1.2347827e-02, 8.6269727e-05, 6.4331929e-05, ..., 1.3470476e-07,
        3.2594623e-03, 1.5846948e-05],
       [1.4650276e-04, 2.9536818e-06, 2.8224284e-04, ..., 1.5846592e-07,
        1.5289562e-04, 3.0337292e-06],
       [2.0218946e-03, 3.7127553e-08, 9.9626786e-01, ..., 2.4969133e-08,
        5.5308128e-06, 1.1835609e-06]], dtype=float32)

In [26]:
predictions=np.argmax(y_hat,axis=1)
predictions

array([4, 0, 9, ..., 6, 6, 2])

In [27]:
test.to_csv("Submission.csv",columns=['filename'],index=False)

In [28]:
sub=pd.read_csv('sample_submission.csv')
sub.head()

,filename,label
0,49000.png,0
1,49001.png,0
2,49002.png,0
3,49003.png,0
4,49004.png,0


In [29]:
test['label']=predictions

In [30]:
test.to_csv("Submission.csv",columns=['filename','label'],index=False)